In [118]:
import os
import openai
import csv
import pandas as pd
import time
from first_pass import HelperFunctions as help
from first_pass import GatherCommodities as gather
from settings import API_KEY, LIBRARY_ID, LIBRARY_TYPE, ZOLTERO_KEY 


client = openai.OpenAI(api_key = API_KEY)

## Commodities Extraction
This will work as the first pass that goes through each of the files and we want to identify all the commodities in each file. We will use chatGPT to do this and then create a master list.

Notes From Goran
- idea: do two commodities --> get the ones mentioned directly for mineral inventory & then another grand list


## Test with 10 and see the cost
Should then try to set it up in parallel

In [122]:

t = time.time()
download_dir = "./reports/old_copper/"
csv_output_path = "./metadata/phase_one_copper_top10percent.csv"
gather.run(download_dir, csv_output_path)
print("\n-------------------------------------------------------------------------\n")
print(f'Evaluation Took: {time.time()-t}')

Working on File: NI 43-101 Technical Report for the Chu Chua Project in North America dated September 2013.pdf file num: 1 out of 79

Response: Yes.
File was correctly uploaded 


{
  "commodities": ["copper", "zinc", "silver", "gold"]
}
Here are the extracted commodities: {'commodities': ['copper', 'zinc', 'silver', 'gold']}
Finished writing row for NI 43-101 Technical Report for the Chu Chua Project in North America dated September 2013.pdf 

Working on File: NI 43-101 Technical Report for the Red Chris project in Canada dated June, 2021.pdf file num: 2 out of 79

Response: YES
File was correctly uploaded 


{
  "commodities": ["copper", "gold", "silver", "arsenic", "antimony", "mercury", "sulphur"]
}
Here are the extracted commodities: {'commodities': ['copper', 'gold', 'silver', 'arsenic', 'antimony', 'mercury', 'sulphur']}
Finished writing row for NI 43-101 Technical Report for the Red Chris project in Canada dated June, 2021.pdf 

Working on File: NI 43-101 Technical Report for t

JSON error for checking the format: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)



{ 'commodities': ['copper', 'gold', 'molybdenum']}
Here are the extracted commodities: {'commodities': ['copper', 'gold', 'molybdenum']}
Finished writing row for NI 43-101 Technical Report for the Donnelly-GJ Project in North America dated April 2016.pdf 

Working on File: NI 43-101 Technical Report for the Wellgreen Project in North America dated April 2011.pdf file num: 5 out of 79

Response: YES
File was correctly uploaded 


{
  "commodities": [
    "nickel", 
    "copper", 
    "platinum", 
    "palladium", 
    "cobalt", 
    "rhodium", 
    "ruthenium", 
    "rhenium", 
    "iridium", 
    "osmium"
  ]
}
Here are the extracted commodities: {'commodities': ['nickel', 'copper', 'platinum', 'palladium', 'cobalt', 'rhodium', 'ruthenium', 'rhenium', 'iridium', 'osmium']}
Finished writing row for NI 43-101 Technical Report for the Wellgreen Project in North America dated April 2011.pdf 

Working on File: NI 43-101 Technical Report for the Trout Bay Project in North America dated July

JSON error for checking the format: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)



{'commodities': ['nickel', 'copper', 'platinum-group elements', 'gold', 'zinc', 'silver', 'lead', 'rhodium', 'palladium']}
Here are the extracted commodities: {'commodities': ['nickel', 'copper', 'platinum-group elements', 'gold', 'zinc', 'silver', 'lead', 'rhodium', 'palladium']}
Finished writing row for NI 43-101 Technical Report for the Raglan Project in North America dated April 2003.pdf 

Working on File: NI 43-101 Technical Report for the Thunder Bay North project in Canada dated December, 2021.pdf file num: 43 out of 79

Response: YES
File was correctly uploaded 


{
  "commodities": [
    "gold",
    "silver",
    "copper",
    "nickel",
    "cobalt",
    "platinum",
    "palladium"
  ]
}
Here are the extracted commodities: {'commodities': ['gold', 'silver', 'copper', 'nickel', 'cobalt', 'platinum', 'palladium']}
Finished writing row for NI 43-101 Technical Report for the Thunder Bay North project in Canada dated December, 2021.pdf 

Working on File: NI 43-101 Technical Report

JSON error for checking the format: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)



{'commodities': ['copper', 'molybdenum']}
Here are the extracted commodities: {'commodities': ['copper', 'molybdenum']}
Finished writing row for NI 43-101 Technical Report for the Gibraltar Project in North America dated June 2011.pdf 

Working on File: NI 43-101 Technical Report for the Parkin Project in North America dated March 2002.pdf file num: 52 out of 79

Response: Yes
File was correctly uploaded 


{
  "commodities": ["copper", "nickel", "platinum", "palladium", "gold", "cobalt"]
}
Here are the extracted commodities: {'commodities': ['copper', 'nickel', 'platinum', 'palladium', 'gold', 'cobalt']}
Finished writing row for NI 43-101 Technical Report for the Parkin Project in North America dated March 2002.pdf 

Working on File: NI 43-101 Technical Report for the Halfmile Lake Project in North America dated July 2009.pdf file num: 53 out of 79

Response: YES
File was correctly uploaded 


{
  "commodities": ["zinc", "copper", "lead", "silver"]
}
Here are the extracted commoditie

## In case you have to move values

In [54]:
# done_df = pd.read_csv("./metadata/phase_one_mvt_zinc_top10percent.csv")
# done_df.head()

,File Name,record_id,Identified Commodities
0,NI 43-101 Technical Report for the Gold Basin ...,0281c869068818160dfea62039277a05196be0e3c3993a...,"gold, silver, copper, lead, zinc, mercury"
1,NI 43-101 Technical Report for the Soledad Mou...,0264ce3186e07cae1162d1f75368ebaa6b501609a1952a...,"gold, silver"
2,NI 43-101 Technical Report for the Torlon Hill...,02113ca702c46a53209531854d1982773cd162418ebdf8...,"zinc, lead, silver"
3,NI 43-101 Technical Report for the Boundary Fa...,02ae35f9cd777fa9e42a3788256e8de8543702a95a94e5...,"gold, silver, lead, zinc, copper"
4,NI 43-101 Technical Report for the McGarry Pro...,02ccb5d5602155759ce42427c5978bece96d63bd71d574...,NaN


In [66]:
# import os
# import shutil

# file_names = []

# for i, row in done_df.iterrows():
#     record_id = row['record_id']
#     fileName = row['File Name']
#     file_names.append(f"{record_id}_{fileName}")


# # Path to the folder where you want to move files from
# folder_path = './reports/mvt_zinc/'

# # Path to the destination folder where you want to move files
# destination_path = './reports/'

# # Iterate through files in the folder
# for filename in os.listdir(folder_path):
#     # Check if the filename matches any in the list
#     if filename in file_names:
#         # Construct the full paths for source and destination
#         source_file = os.path.join(folder_path, filename)
#         destination_file = os.path.join(destination_path, filename)
        
#         # Move the file to the destination folder
#         shutil.move(source_file, destination_file)
#         print(f"Moved {filename} to {destination_path}")

# print("All files moved successfully.")